In [3]:
import pandas as pd

data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [4]:
print(len(documents))
print(documents[:5])

1103663
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


We have to perform the following steps:
1. Tokenization- Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
2. Words that have fewer than 3 characters are removed
3. All stopwords are removed ("a", "the", "an")
4. Words are lemmatized- words in third person are changed to first person and verbs in past and future tenses are changed into present tense
5. Words are stemmed - words are reduced to their root form

In [7]:
# Import libraries
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)


In [8]:
import nltk

In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/nirmalb/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [21]:
stemmer = PorterStemmer()

In [22]:
# Function to lemmatize and stem preprocess steps on the dataset. 
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [23]:
# Example of preprocessing

doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [24]:
# We are going to preprocess the headline text, saving the results as 'processed documents'
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [25]:
# Create a dictionary containing the word frequency in the training set 

dictionary = gensim.corpora.Dictionary(processed_docs) # initialize the dictionary
count = 0
print(dictionary)
for k,v in dictionary.iteritems():
    print(k,v)
    count += 1
    if count > 10:
        break


Dictionary(62293 unique tokens: ['broadcast', 'commun', 'decid', 'licenc', 'awar']...)
0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [27]:
# Now, we are going to filter out tokens that appear in a. less than 15 documents or b. more than 50% documents, c. then from there only keep the 1 mil most frequent tokens
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [28]:
# Now, for each document that was initially present, we are going to create a dictionary reporting how many words
# and how many times these words appear. Save this to 'bow_corpus', then check our selected document earlier. 

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (484, 1), (4022, 1)]

In [31]:
# Preview the bag of words for our sample document that we printed above

bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], dictionary[bow_doc_4310[i][0]], bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 484 ("rain") appears 1 time.
Word 4022 ("dampen") appears 1 time.


In [32]:
# Create a TFIDF model to count the words that are key differentiators but also appear often enough

from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5903873663261261),
 (1, 0.38522586675616677),
 (2, 0.49747840165121515),
 (3, 0.5055284652272439)]
